# Bitget Earn Trading Library Demo

This notebook demonstrates how to use the `bitget_earn_trading` library to integrate Bitget's Earn (Flexible Savings) products with Spot and Futures trading.

## Features

- **Earn Integration**: Automatically manage flexible savings deposits and withdrawals
- **Spot Trading**: Buy from savings and sell to savings seamlessly
- **Futures Trading**: Enter and exit positions using savings funds
- **Automatic Precision**: Handles price and quantity precision automatically
- **Rate Limiting**: Built-in rate limiter (10 requests/second) to respect API limits
- **Error Handling**: Comprehensive error handling for API calls

## Table of Contents

1. [Credentials Setup](#1-credentials-setup)
2. [Earn Operations](#2-earn-operations)
3. [Spot Trading](#3-spot-trading)
4. [Futures Trading](#4-futures-trading)


In [ ]:
import os
import time
from dotenv import load_dotenv

# Force reload environment variables from .env fileter
load_dotenv(override=True)

## 1. Credentials Setup

⚠️ **Important**: Keep your API keys private! Anyone with these keys can control your trading account and access your funds.

In [ ]:
PASSPHRASE = os.environ.get('BITGET_PASSPHRASE')
API_KEY = os.getenv('BITGET_API_KEY')
API_SECRET = os.getenv('BITGET_API_SECRET')

## 2. Earn Operations

In [ ]:
from bitget_earn_trading import BitgetEarn

earn_client = BitgetEarn(API_KEY, API_SECRET, PASSPHRASE)

### 2.1 Check Current Savings Balance

In [ ]:
coin = "USDT"
current_balance = earn_client.get_flexible_savings_amount(coin)

print(f"💰 Current {coin} in flexible savings: {current_balance}")

### 2.2 Subscribe to Savings (Deposit)

In [ ]:
coin = "USDT"
deposit_amount = 20

try:
    print(f"📥 Depositing {deposit_amount} {coin} to flexible savings...")
    
    earn_client.subscribe_savings(coin, deposit_amount)
    
    print(f"✅ Deposit successful!")
    
    time.sleep(2)
    current_balance = earn_client.get_flexible_savings_amount(coin)

    print(f"💰 Current {coin} in flexible savings: {current_balance}")
    
except Exception as e:
    print(f"❌ Deposit failed: {e}")


### 2.3 Redeem from Savings (Withdraw)

In [ ]:
coin = "USDT"
withdraw_amount = 10

try:
    print(f"📤 Withdrawing {withdraw_amount} {coin} from flexible savings...")
    
    earn_client.redeem_savings(coin, withdraw_amount) 
    
    print(f"✅ Withdrawal successful!")

    time.sleep(2)
    updated_balance = earn_client.get_flexible_savings_amount(coin)

    print(f"💰 Updated {coin} in flexible savings: {updated_balance}")

except Exception as e:
    print(f"❌ Withdrawal failed: {e}")

## 3. Spot Trading

The Spot Trading module integrates savings with spot trading operations, allowing you to buy directly from savings and sell directly to savings.


In [ ]:
from bitget_earn_trading import BitgetEarnSpot

spot_client = BitgetEarnSpot(API_KEY, API_SECRET, PASSPHRASE)

### 3.1 Check Spot Account Balance


In [ ]:
usdt_balance = spot_client.get_balance("USDT")
btc_balance = spot_client.get_balance("BTC")
savings_balance = spot_client.get_flexible_savings_amount("USDT")

print(f"💰 Spot Account Balances:")
print(f"   USDT: {usdt_balance.get('available', '0')} (available)")
print(f"   BTC: {btc_balance.get('available', '0')} (available)")

print(f"💳 Savings Balance:")
print(f"   USDT in Savings: {savings_balance}")


### 3.2 Buy from Savings

This operation:
1. Redeems USDT from flexible savings
2. Waits for the redemption to complete
3. Places a buy order for the specified cryptocurrency


In [ ]:
symbol = "BTCUSDT"
quote_coin = "USDT"
buy_amount = 10

try:
    print(f"🛒 Buying {buy_amount} {quote_coin} worth of {symbol} from savings...")
    
    result = spot_client.buy_from_savings(
        symbol=symbol,
        quote_coin=quote_coin,
        amount=buy_amount,
        order_type="market" 
    )
    
    print(f"✅ Buy order completed!")
    print(f"📄 Redemption Order ID: {result['redeem_order_id']}")
    print(f"📄 Buy Order ID: {result['buy_order_id']}")
    
except Exception as e:
    print(f"❌ Buy from savings failed: {e}")


### 3.3 Sell to Savings

This operation:
1. Places a sell order for the specified cryptocurrency
2. Waits for the order to complete
3. Automatically deposits the proceeds to flexible savings


In [ ]:
symbol = "BTCUSDT"
quote_coin = "USDT"
sell_amount = 0.000092542

try:
    print(f"💰 Selling {sell_amount} {symbol} and depositing to savings...")
    
    result = spot_client.sell_to_savings(
        symbol=symbol,
        amount_base=sell_amount,
        quote_coin=quote_coin,
        order_type="market"
    )
    
    print(f"✅ Sell to savings completed!")
    print(f"📄 Sell Order ID: {result['sell_order_id']}")
    
    if result['deposit_order_id']:
        print(f"📄 Savings Deposit Order ID: {result['deposit_order_id']}")
    else:
        print("⚠️ No proceeds deposited to savings (order may not have filled)")
        
except Exception as e:
    print(f"❌ Sell to savings failed: {e}")


### 3.4 Advanced: Limit Orders

You can also place limit orders instead of market orders for better price control.


In [ ]:
symbol = "BTCUSDT"
quote_coin = "USDT"
buy_amount = 10
limit_price = 90000

try:
    print(f"🎯 Placing limit buy order for {symbol} at ${limit_price}...")
    
    result = spot_client.buy_from_savings(
        symbol=symbol,
        quote_coin=quote_coin,
        amount=buy_amount,
        order_type="limit",
        price=limit_price,
    )
    
    print(f"✅ Limit buy order placed!")
    print(f"📄 Order ID: {result['buy_order_id']}")
    
except Exception as e:
    print(f"❌ Limit buy order failed: {e}")


In [ ]:
symbol = "BTCUSDT"
quote_coin = "USDT"
buy_amount = 10
limit_price = 90000

try:
    print(f"🎯 Placing limit buy order for {symbol} at ${limit_price}...")
    
    result = spot_client.buy_from_savings(
        symbol=symbol,
        quote_coin=quote_coin,
        amount=buy_amount,
        order_type="limit",
        price=limit_price,
    )
    
    print(f"✅ Limit buy order placed!")
    print(f"📄 Order ID: {result['buy_order_id']}")
    
except Exception as e:
    print(f"❌ Limit buy order failed: {e}")


## 4. Futures Trading

The Futures Trading module allows you to enter and exit positions using funds from your flexible savings account.


In [ ]:
from bitget_earn_trading import BitgetEarnFutures

futures_client = BitgetEarnFutures(API_KEY, API_SECRET, PASSPHRASE)

### 4.1 Check Futures Account Balance

In [ ]:
futures_balance = futures_client.get_balance("USDT")
savings_balance = futures_client.get_flexible_savings_amount("USDT")

print(f"💰 Futures Account Balance:")
print(f"   USDT Available: {futures_balance.get('available', '0')}")
print(f"   USDT Locked: {futures_balance.get('locked', '0')}")
print(f"   USDT Equity: {futures_balance.get('equity', '0')}")

print(f"💳 Savings Balance:")
print(f"   USDT in Savings: {savings_balance}")

### 4.2 Enter Long Position from Savings

This operation:
1. Redeems USDT from flexible savings
2. Transfers the funds from spot to futures account
3. Opens a long position with the specified parameters


In [ ]:
symbol = "ADAUSDT"
margin_coin = "USDT"
position_size = 10
side = "long"

try:
    print(f"📈 Opening {side} position for {symbol} using {position_size} {margin_coin} from savings...")
    
    result = futures_client.enter_position_from_savings(
        symbol=symbol,
        margin_coin=margin_coin,
        amount=position_size,
        side=side,
        order_type="market",
        margin_mode="isolated" 
    )
    
    print(f"✅ Long position opened successfully!")
    print(f"📄 Redemption Order ID: {result['redeem_order_id']}")
    print(f"📄 Position Order ID: {result['position_order_id']}")
    
except Exception as e:
    print(f"❌ Failed to open long position: {e}")


### 4.3 Enter Short Position from Savings

Similar to long positions, but opens a short position instead.


In [ ]:
symbol = "ADAUSDT"
margin_coin = "USDT"
position_size = 10  # in quote currency
side = "short"

try:
    print(f"📉 Opening {side} position for {symbol} using {position_size} {margin_coin} from savings...")
    
    result = futures_client.enter_position_from_savings(
        symbol=symbol,
        margin_coin=margin_coin,
        amount=position_size,
        side=side,
        order_type="market",
        margin_mode="crossed"
    )
    
    print(f"✅ Short position opened successfully!")
    print(f"📄 Redemption Order ID: {result['redeem_order_id']}")
    print(f"📄 Position Order ID: {result['position_order_id']}")
    
except Exception as e:
    print(f"❌ Failed to open short position: {e}")


### 4.4 Exit Position to Savings

This operation:
1. Closes the specified position
2. Transfers the remaining funds from futures to spot account
3. Deposits the proceeds to flexible savings


In [ ]:
symbol = "ADAUSDT"
position_size = 34  # in base currency
side = "long"  
margin_coin = "USDT"

try:
    print(f"🔄 Closing {side} position for {symbol} and depositing to savings...")
    
    result = futures_client.exit_position_to_savings(
        symbol=symbol,
        size=position_size,
        side=side,
        margin_coin=margin_coin,
        order_type="market"
    )
    
    print(f"✅ Position closed and funds deposited to savings!")
    print(f"📄 Close Order ID: {result['close_order_id']}")
    
    if result['deposit_order_id']:
        print(f"📄 Savings Deposit Order ID: {result['deposit_order_id']}")
    else:
        print("⚠️ No funds deposited to savings (no available balance)")
        
except Exception as e:
    print(f"❌ Failed to close position: {e}")

### 4.5 Advanced: Limit Orders for Futures

You can also use limit orders for more precise entry and exit points.


In [ ]:
symbol = "ADAUSDT"
margin_coin = "USDT"
position_size = 10 
side = "long"
limit_price = 0.4

try:
    print(f"🎯 Placing limit long order for {symbol} at ${limit_price}...")
    
    result = futures_client.enter_position_from_savings(
        symbol=symbol,
        margin_coin=margin_coin,
        amount=position_size,
        side=side,
        order_type="limit",
        price=limit_price,
        force="gtc"
    )
    
    print(f"✅ Limit long order placed!")
    print(f"📄 Position Order ID: {result['position_order_id']}")
    
except Exception as e:
    print(f"❌ Failed to place limit long order: {e}")
